## Analysis of FFPE Adenocarcinoma Prostate Dataset


In [1]:
import os,csv,re
import pandas as pd
import numpy as np
import numba
import scanpy as sc
import math
import SpaGCN as spg
from scipy.sparse import issparse
import random, torch
import warnings
warnings.filterwarnings("ignore")
import matplotlib.colors as clr
import matplotlib.pyplot as plt
import SpaGCN as spg
#In order to read in image data, we need to install some package. Here we recommend package "opencv"
#inatll opencv in python
#!pip3 install opencv-python
import cv2
print(spg.__version__)

1.2.0


In [2]:
##Read in preprocessed adata
#Read in gene expression and spatial location
adata=sc.read("ffpe_human_prostate_cancer.h5ad")

#read in spatial positions
spatial=pd.read_csv("FFPE_Visium_Human_ProstateCancer/spatial/tissue_positions_list.csv",sep=",",header=None,na_filter=False,index_col=0,names=["barcode", "in_tissue", "array_row", "array_col","pxl_row_in_fullres","pxl_col_in_fullres"]) 

#Read in hitology image
img=cv2.imread("FFPE_Visium_Human_ProstateCancer/spatial/tissue_hires_image.tiff")


In [3]:
##re index spatial table according to barcodes in adata
sorted_barcodes = list(adata.obs_names)

spatial = spatial.reindex(sorted_barcodes)

In [4]:
#Set coordinates
x_array=adata.obs["array_row"].tolist()
y_array=adata.obs["array_col"].tolist()
x_pixel=spatial["pxl_row_in_fullres"].tolist()
y_pixel=spatial["pxl_col_in_fullres"].tolist()

print(img)
#Test coordinates on the image
img_new=img.copy()
for i in range(len(x_pixel)):
    x=x_pixel[i]
    y=y_pixel[i]
    img[int(x-20):int(x+20), int(y-20):int(y+20),:]=0

cv2.imwrite('ffpe_prostate_cancer_scanpy_processed_image_map.jpg', img_new)

[[[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [251 252 250]
  [251 252 250]
  [252 252 252]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [250 251 249]
  [250 251 249]
  [251 251 250]]

 [[255 255 255]
  [255 255 255]
  [255 255 255]
  ...
  [250 251 248]
  [250 251 249]
  [250 251 250]]

 ...

 [[254 254 254]
  [254 254 254]
  [254 254 254]
  ...
  [255 255 255]
  [255 255 255]
  [254 254 254]]

 [[254 254 254]
  [254 254 254]
  [254 254 254]
  ...
  [254 254 254]
  [254 254 254]
  [253 253 253]]

 [[254 254 254]
  [254 254 254]
  [254 254 254]
  ...
  [254 254 254]
  [254 254 254]
  [252 252 252]]]


True

In [5]:
#Calculate adjacent matrix
s=1
b=49
#print(img)
#adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, x_pixel=x_pixel, y_pixel=y_pixel, image=img, beta=b, alpha=s, histology=True)
#If histlogy image is not available, SpaGCN can calculate the adjacent matrix using the fnction below
adj=spg.calculate_adj_matrix(x=x_pixel,y=y_pixel, histology=False)
print(adj)
np.savetxt('ffpe_prostate_cancer_adj.csv', adj, delimiter=',')

Calculateing adj matrix using xy only...
[[    0.     12252.916   9160.963  ... 10745.774  10952.932  13136.357 ]
 [12252.916      0.     15729.188  ...  1615.2554  3719.4866 13922.7295]
 [ 9160.963  15729.188      0.     ... 14867.287  12478.298   6484.244 ]
 ...
 [10745.774   1615.2554 14867.287  ...     0.      3811.2217 13721.165 ]
 [10952.932   3719.4866 12478.298  ...  3811.2217     0.     10204.537 ]
 [13136.357  13922.7295  6484.244  ... 13721.165  10204.537      0.    ]]


In [6]:
#set hyper-parameters
p=0.5 
#Find the l value given p
l=spg.search_l(p, adj, start=0.01, end=1000, tol=0.01, max_run=100)
print(l)

Run 1: l [0.01, 1000], p [0.0, 70.7492878891595]
Run 2: l [0.01, 500.005], p [0.0, 17.97595977783203]
Run 3: l [0.01, 250.0075], p [0.0, 3.945427417755127]
Run 4: l [125.00874999999999, 250.0075], p [0.3675481081008911, 3.945427417755127]
Run 5: l [125.00874999999999, 187.508125], p [0.3675481081008911, 1.8351752758026123]
Run 6: l [125.00874999999999, 156.2584375], p [0.3675481081008911, 1.0038671493530273]
Run 7: l [125.00874999999999, 140.63359375], p [0.3675481081008911, 0.6553142070770264]
recommended l =  132.821171875
132.821171875


In [7]:
#Going with 14 clusters as this is what I got from previous data - this has 0 index
n_clusters=14
#Set seed
r_seed=t_seed=n_seed=100
#Search for suitable resolution
res=spg.search_res(adata, adj, l, n_clusters, start=0.7, step=0.1, tol=5e-3, lr=0.05, max_epochs=20, r_seed=r_seed, t_seed=t_seed, n_seed=n_seed)

Start at res =  0.7 step =  0.1
Initializing cluster centers with louvain, resolution =  0.7
Epoch  0
Epoch  10
Res =  0.7 Num of clusters =  10
Initializing cluster centers with louvain, resolution =  0.7999999999999999
Epoch  0
Epoch  10
Res =  0.7999999999999999 Num of clusters =  10
Res changed to 0.7999999999999999
Initializing cluster centers with louvain, resolution =  0.8999999999999999
Epoch  0
Epoch  10
Res =  0.8999999999999999 Num of clusters =  12
Res changed to 0.8999999999999999
Initializing cluster centers with louvain, resolution =  0.9999999999999999
Epoch  0
Epoch  10
Res =  0.9999999999999999 Num of clusters =  16
Step changed to 0.05
Initializing cluster centers with louvain, resolution =  0.95
Epoch  0
Epoch  10
Res =  0.95 Num of clusters =  16
Step changed to 0.025
Initializing cluster centers with louvain, resolution =  0.9249999999999999
Epoch  0
Epoch  10
Res =  0.9249999999999999 Num of clusters =  15
Step changed to 0.0125
Initializing cluster centers with 

In [8]:
clf=spg.SpaGCN()
clf.set_l(l)
#Set seed
random.seed(r_seed)
torch.manual_seed(t_seed)
np.random.seed(n_seed)
#Run
clf.train(adata,adj,init_spa=True,init="louvain",res=res, tol=5e-3, lr=0.05, max_epochs=200)
y_pred, prob=clf.predict()
adata.obs["pred"]= y_pred
adata.obs["pred"]=adata.obs["pred"].astype('category')
#Do cluster refinement(optional)
#shape="hexagon" for Visium data, "square" for ST data.
adj_2d=spg.calculate_adj_matrix(x=x_array,y=y_array, histology=False)
refined_pred=spg.refine(sample_id=adata.obs.index.tolist(), pred=adata.obs["pred"].tolist(), dis=adj_2d, shape="hexagon")
adata.obs["refined_pred"]=refined_pred
adata.obs["refined_pred"]=adata.obs["refined_pred"].astype('category')

Initializing cluster centers with louvain, resolution =  0.9187499999999998
Epoch  0
Epoch  10
Epoch  20
Epoch  30
Epoch  40
Epoch  50
Epoch  60
Epoch  70
Epoch  80
Epoch  90
Epoch  100
Epoch  110
Epoch  120
Epoch  130
Epoch  140
Epoch  150
Epoch  160
Epoch  170
Epoch  180
Epoch  190
Calculateing adj matrix using xy only...


In [9]:
#Plot spatial domains
#adata=sc.read("/Users/natalie.charitakis/Documents/Documents - MCRI-U0197/Work/Projects/PhD Project/Benchmarking SRT/ff_human_breast_cancer/ff_breast_cancer_procesed.h5ad")
#Set colors used
plot_color=["#F56867","#FEB915","#C798EE","#59BE86","#7495D3","#D1D1D1","#6D1A9C","#15821E","#3A84E6","#997273","#787878","#DB4C6C","#9E7A7A","#554236","#AF5F3C","#93796C","#F9BD3F","#DAB370","#877F6C","#268785"]
#Plot spatial domains
domains="pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
#ax=sc.pl.scatter(adata,alpha=1,x="y_pixel",y="x_pixel",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
#change above line so it comes from adata input
ax=sc.pl.scatter(adata,alpha=1,x="array_row",y="array_col",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("ffpe_prostate_cancer_spagcn_pred.png", dpi=600)
plt.close()

#Plot refined spatial domains
domains="refined_pred"
num_celltype=len(adata.obs[domains].unique())
adata.uns[domains+"_colors"]=list(plot_color[:num_celltype])
ax=sc.pl.scatter(adata,alpha=1,x="array_row",y="array_col",color=domains,title=domains,color_map=plot_color,show=False,size=100000/adata.shape[0])
ax.set_aspect('equal', 'box')
ax.axes.invert_yaxis()
plt.savefig("ffpe_prostate_cancer_spagcn_refined_pred.png", dpi=600)
plt.close()

In [10]:
#Use domain 0 as an example
target=0
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 377.03516819571865]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 122.42354740061162]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 42.31039755351682]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 20.129969418960243]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.7782874617737, 20.129969418960243]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.564220183486238
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.564220183486238
 Cluster 0 has neighbors:
Dmain  2 : 

In [11]:
target=1
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 345.05494505494505]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 105.71428571428571]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 35.93846153846154]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 17.193406593406593]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [7.650549450549451, 17.193406593406593]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=10.837362637362638
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 10.837362637362638
 Cluster 1 has neighbors:
Dmain  9 

In [12]:
target=2
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 389.3903743315508]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 121.00534759358288]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 41.36363636363637]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 19.641711229946523]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.564171122994653, 19.641711229946523]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.248663101604278
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.248663101604278
 Cluster 2 has neighbors:
Dmain  0 :

In [13]:
target=3
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 417.9684542586751]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 128.18927444794951]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 42.548895899053626]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 19.965299684542586]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.602523659305994, 19.965299684542586]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.394321766561514
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.394321766561514
 Cluster 3 has neighbors:
Dmain  6 

In [14]:
target=4
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 268.43684210526317]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 91.59473684210526]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 32.56315789473684]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [7.094736842105263, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 6: radius [2.8945670388638973, 3.3880182057619095], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 7: radius [3.1412926223129034, 3.3880182057619095], num_nbr [9.93684

Run 59: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 60: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 61: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 62: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 63: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 64: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy only...
Run 65: radius [3.162277660168379, 3.1622776601683795], num_nbr [9.936842105263159, 15.74736842105263]
Calculateing adj matrix using xy 

TypeError: unsupported operand type(s) for ** or pow(): 'NoneType' and 'int'

In [15]:
target=5
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 410.92083333333335]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 125.22916666666667]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 42.325]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 19.991666666666667]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.6625, 19.991666666666667]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.433333333333334
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.433333333333334
 Cluster 5 has neighbors:
Dmain  2 :  302
Dmain  6 :  267

In [16]:
target=6
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 412.861581920904]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 127.80790960451978]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 43.067796610169495]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 20.274011299435028]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.751412429378531, 20.274011299435028]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.590395480225988
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.590395480225988
 Cluster 6 has neighbors:
Dmain  3 :

In [17]:
target=7
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 323.1864406779661]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 103.22598870056497]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 35.389830508474574]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 17.225988700564972]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [7.68361581920904, 17.225988700564972]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=10.892655367231638
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 10.892655367231638
 Cluster 7 has neighbors:
Dmain  4 :

In [18]:
target=8
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 322.2788844621514]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 108.52988047808765]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 39.47011952191235]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 19.310756972111555]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.605577689243027, 19.310756972111555]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.171314741035857
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.171314741035857
 Cluster 8 has neighbors:
Dmain  2 :

In [19]:
target=9
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 310.40909090909093]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 104.45454545454545]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 38.16161616161616]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 18.707070707070706]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.292929292929292, 18.707070707070706]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=11.792929292929292
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 11.792929292929292
 Cluster 9 has neighbors:
Dmain  1 

In [20]:
target=10
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 208.92307692307693]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 66.98816568047337]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 24.31360946745562]
Calculateing adj matrix using xy only...
recommended radius =  3.3880182057619095 num_nbr=12.21301775147929
3.3880182057619095
radius= 3.3880182057619095 average number of neighbors for each spot is 12.21301775147929
 Cluster 10 has neighbors:
No neighbor domain found, try bigger radius or smaller ratio.
None


TypeError: 'NoneType' object is not subscriptable

In [21]:
target=11
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 419.01785714285717]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 132.76785714285714]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 44.375]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 20.839285714285715]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.946428571428571, 20.839285714285715]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.928571428571429
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.928571428571429
 Cluster 11 has neighbors:
Dmain  4 :  142
[4]

In [22]:
target=12
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 438.5652173913044]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 129.91304347826087]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 42.21739130434783]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 19.73913043478261]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.521739130434783, 19.73913043478261]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.23913043478261
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.23913043478261
 Cluster 12 has neighbors:
No neighbor d

TypeError: 'NoneType' object is not subscriptable

In [23]:
target=13
#Set filtering criterials
min_in_group_fraction=0.8
min_in_out_group_ratio=1
min_fold_change=1.5
#Search radius such that each spot in the target domain has approximately 10 neighbors on average
adj_2d=spg.calculate_adj_matrix(x=x_array, y=y_array, histology=False)
start, end= np.quantile(adj_2d[adj_2d!=0],q=0.001), np.quantile(adj_2d[adj_2d!=0],q=0.1)
r=spg.search_radius(target_cluster=target, cell_id=adata.obs.index.tolist(), x=x_array, y=y_array, pred=adata.obs["pred"].tolist(), start=start, end=end, num_min=10, num_max=14,  max_run=100)
#Detect neighboring domains
print(r)
nbr_domians=spg.find_neighbor_clusters(target_cluster=target,
                                   cell_id=adata.obs.index.tolist(), 
                                   x=adata.obs["array_row"].tolist(), 
                                   y=adata.obs["array_col"].tolist(), 
                                   pred=adata.obs["pred"].tolist(),
                                   radius=r,
                                   ratio=1/2)


print(nbr_domians)
nbr_domians=nbr_domians[0:3]

de_genes_info=spg.rank_genes_groups(input_adata=adata,
                                target_cluster=target,
                                nbr_list=nbr_domians, 
                                label_col="pred", 
                                adj_nbr=True, 
                                log=True)
#Filter genes
de_genes_info=de_genes_info[(de_genes_info["pvals_adj"]<0.05)]
filtered_info_domain_0=de_genes_info
filtered_info_domain_0=filtered_info_domain_0[(filtered_info_domain_0["pvals_adj"]<0.05) &
                            (filtered_info_domain_0["in_out_group_ratio"]>min_in_out_group_ratio) &
                            (filtered_info_domain_0["in_group_fraction"]>min_in_group_fraction) &
                            (filtered_info_domain_0["fold_change"]>min_fold_change)]
filtered_info_domain_0=filtered_info_domain_0.sort_values(by="in_group_fraction", ascending=False)
filtered_info_domain_0["target_dmain"]=target
filtered_info_domain_0["neighbors"]=str(nbr_domians)
print("SVGs for domain ", str(target),":", filtered_info_domain_0["genes"].tolist())

Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Calculateing adj matrix using xy only...
Run 1: radius [1.4142135381698608, 17.20465087890625], num_nbr [1.0, 407.17045454545456]
Calculateing adj matrix using xy only...
Run 2: radius [1.4142135381698608, 9.309432208538055], num_nbr [1.0, 127.56818181818181]
Calculateing adj matrix using xy only...
Run 3: radius [1.4142135381698608, 5.361822873353958], num_nbr [1.0, 43.11363636363637]
Calculateing adj matrix using xy only...
Run 4: radius [1.4142135381698608, 3.3880182057619095], num_nbr [1.0, 20.386363636363637]
Calculateing adj matrix using xy only...
Run 5: radius [2.401115871965885, 3.3880182057619095], num_nbr [8.818181818181818, 20.386363636363637]
Calculateing adj matrix using xy only...
recommended radius =  2.8945670388638973 num_nbr=12.704545454545455
2.8945670388638973
radius= 2.8945670388638973 average number of neighbors for each spot is 12.704545454545455
 Cluster 13 has neighbors:
Dmain  5